In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn

import numpy

In [0]:
class NeuralNetwork(nn.Module):

    def __init__(self, inodes, hnodes, onodes, learning_rate):
        super().__init__()
        
        self.inodes = inodes
        self.hnodes = hnodes
        self.onodes = onodes
        
       
        self.lr = learning_rate
 
        self.linear_ih = nn.Linear(inodes, hnodes, bias=False)
        self.linear_ho = nn.Linear(hnodes, onodes, bias=False)
        
        self.activation = nn.Sigmoid()
        self.error_function = torch.nn.MSELoss(size_average=False)
        self.optimiser = torch.optim.SGD(self.parameters(), self.lr)

        pass

    
    def forward(self, inputs_list):
        inputs = Variable(torch.FloatTensor(inputs_list).view(1, self.inodes))
        
        hidden_inputs = self.linear_ih(inputs)
        hidden_outputs = self.activation(hidden_inputs)
        final_inputs = self.linear_ho(hidden_outputs)
        final_outputs = self.activation(final_inputs)
        
        return final_outputs

    
    def train(self, inputs_list, targets_list):
    
        output = self.forward(inputs_list)
        target_variable = Variable(torch.FloatTensor(targets_list).view(1, self.onodes), requires_grad=False)
     
        loss = self.error_function(output, target_variable)
        self.optimiser.zero_grad()
        loss.backward()
        self.optimiser.step()
        pass

    pass

In [0]:
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

learning_rate = 0.1
n = NeuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)





/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


NeuralNetwork(
  (linear_ih): Linear(in_features=784, out_features=200, bias=False)
  (linear_ho): Linear(in_features=200, out_features=10, bias=False)
  (activation): Sigmoid()
  (error_function): MSELoss()
)

In [0]:
training_data_file = open("/content/drive/My Drive/mnist/mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [0]:
 %%timeit

epochs = 5

for e in range(epochs):
    for record in training_data_list:
        
        all_values = record.split(',')
    
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
      
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

1 loop, best of 3: 5min 21s per loop


In [0]:

test_data_file = open("/content/drive/My Drive/mnist/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [0]:
scorecard = []


for record in test_data_list:
    all_values = record.split(',')
  
    correct_label = int(all_values[0])
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    outputs = n.forward(inputs)
    m, label = outputs.max(1)
    if (label.data[0] == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass
    
    pass

In [0]:

scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.9833
